In [1]:
from preprocessing import contents_clean, contents_separation, distribution
from preprocessing.morphanal import analyze_corpus
import pandas as pd
from tqdm import tqdm

In [4]:
df = pd.read_csv("a.csv")

In [5]:
df['clean'] = df['본문'].apply(lambda x : contents_clean(x))

In [6]:
df['sep'] = df['clean'].apply(lambda x : contents_separation(x))

In [7]:
df2 = pd.DataFrame(columns = ['str', 'idx'])
for idx, lst in tqdm(enumerate(df['sep'].tolist())):
    split_df= distribution(idx,lst)
    df2 = pd.concat([df2,split_df])
else:
    df2.dropna(axis=0, inplace=True)
    df2.reset_index(drop=True, inplace=True)
df2

50it [00:00, 1193.60it/s]


,str,idx
0,7월 개봉 예정 영화 신작 영화추천 신작 영화 추천 극장 개봉 예정 영화 7월 리부...,0
1,문제는 쥬라기 월드 탑건 신비한 동물사전 레전드 영화의 리부트가 연일 관객들의 기대...,0
2,어린 시절의 향수를 진하게 불러주는 오마주는 반갑지만 그때 느꼈던 감동과 재미는 다...,0
3,저야 극장에서 보는 어지간한 영화는 재밌게 보고 나오는 편입니다만 그래서 극장 관람...,0
4,그래서 2차 3차 가공해서 영화를 해석하고 추천하는 글을 쓰는 데는 생각이 참 많아...,0
...,...,...
2426,왜냐하면 저 위로 올라가야하는데 아이혼자 클라이밍으로 올라가서 높은 곳을 걸어야하거든요,49
2427,저희 아이는 1시간동안 땀이 뻘뻘 나도록 미끄럼틀을 오르내렸어요,49
2428,저 혹등고래 입속으로 들어가면 피노키오 인형이 있답니다,49
2429,두빛나래와 국립해양생물자원관은 장항에 위치해서 하루 여행 코스로 좋은 것 같아요,49


In [8]:
import pickle
# 데이터프레임을 피클 파일로 저장
with open('b.pkl', 'wb') as file:
    pickle.dump(df2, file)

In [ ]:
# 저장된 피클 파일을 불러오기
with open('b.pkl', 'rb') as file:
    df2 = pickle.load(file)
df2

In [10]:
noun_df, verb_df, adj_df, josa_df, df3 = analyze_corpus(df2)

100%|██████████| 2431/2431 [00:27<00:00, 86.92it/s] 

2431 [['7월', '개봉', '예정', '영화', '신작', '영화', '추천', '신작', '영화', '추천', '극장', '개봉', '예정', '영화', '7월', '리부트', '영화', '가', '어느덧', '거대한', '상업', '영화', '시장', '에', '한자리', '크게', '잡고있다'], ['문제', '는', '쥬라기', '월드', '탑건', '신비한', '동물', '사전', '레전드', '영화', '의', '리부트', '가', '연일', '관객', '들', '의', '기대', '와', '실망', '을', '적잖이', '섞어서', '주고', '있다는', '점', '이', '죠'], ['어린', '시절', '의', '향수', '를', '진하게', '불러주는', '오마주', '는', '반갑지만', '그때', '느꼈던', '감동', '과', '재미', '는', '다른', '이야기', '다', '범죄', '도시', '마녀', '등', '시리즈', '영화', '역시', '관객', '들', '의', '기대했던', '마음', '을', '촉촉하게', '적셔', '주기도', '또는', '더', '큰', '실망', '감', '을', '안겨', '주기도', '하는', '것', '이', '지금', '의', '영화', '시장', '인', '듯', '보입니다'], ['저', '야', '극장', '에서', '보는', '어지간한', '영화', '는', '재밌게', '보고', '나오는', '편입', '니', '다만', '그래서', '극장', '관람', '이', '신중하죠'], ['그래서', '2', '차', '3', '차', '가공', '해서', '영화', '를', '해석', '하고', '추천', '하는', '글', '을', '쓰는', '데', '는', '생각', '이', '참', '많아지는', '요즘', '입니다'], ['가장', '안타까웠던', '점', '은', '초', '호화', '캐스팅', '으로', '화제', '였던', '거장', '고레', '에다', '히로'